### collaborative filtering

#### item based

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [111]:
import random
rd = np.random.RandomState(0)
data = np.random.randint(0, 2, [200, 50]) # 客户*商品矩阵

In [112]:
import scipy.sparse
dataSp = scipy.sparse.csr_matrix(data) # 转为稀疏矩阵便于存储计算

#### 基于客户的协同过滤算法
计算客户之间的相似度，将和该用户最相似的用户喜欢的物品推荐给该用户

In [113]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

usim = cosine_similarity(dataSp, dense_output=False) # 计算客户之间的相似度

In [117]:
# 针对目标用户A，找到S个相似的人，并推荐K个商品
S, K = 10, 5
userNum = dataSp.shape[0] 
rec = {}
for user in range(0, userNum): 
    usimi = [usim[0, i] for i in range(0, userNum)]
    sorted_id = sorted(range(len(usimi)), key=lambda k: usimi[k], reverse=True)
    usims = sorted_id[1:S+1]    # 找到最相似的S个人

    item_rate = np.dot(usim[user, usims].toarray(), dataSp[usims, :].toarray())[0]  # 用户对每个商品的喜好程度
    sorted_rate = sorted(range(len(item_rate)), key=lambda k: item_rate[k], reverse=True)  
    # sorted_rate2 = [i for i in sorted_rate if dataSp[user, i] <= 0][0:K] # 没用过的商品排序选K个
    sorted_rate2 = sorted_rate[0:K] # 排序选K个
    
    rec[user] = sorted_rate2


In [123]:
### 推荐类模型监控
print("Evaluation start ...")
N = dataSp.shape[1]
# 准确率和召回率
hit = 0
rec_count = 0
test_count = 0
# 覆盖率
all_rec_movies = set()

for tusr in range(150, dataSp.shape[0]): # 测试集样本为后50客户
    test_items = [i for i in range(0, dataSp.shape[1]) if dataSp[tusr, i] > 0]
    rec_items = rec[tusr]

    for item in rec_items:
        if item in test_items:
            hit += 1
        all_rec_movies.add(item)

    rec_count += N
    test_count += len(test_items)

precision = hit / (1.0 * rec_count)
recall = hit / (1.0 * test_count)
coverage = len(all_rec_movies) / (1.0 * dataSp.shape[1])
print('precisioin=%.4f\trecall=%.4f\tcoverage=%.4f' % (precision, recall, coverage))

Evaluation start ...
precisioin=0.0608	recall=0.1216	coverage=0.2000


#### 基于物品的协同过滤算法
只需要把data矩阵转置